## Data Analysis of European Soccer Database with SQL 

In [1]:
# importing the necessary libraries
import pandas as pd 
import sqlite3 

In [2]:
# connecting the database
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [3]:
# printing all the tables present in database.sqlite 
for row in c.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    print(list(row))

['sqlite_sequence']
['Player_Attributes']
['Player']
['Match']
['League']
['Country']
['Team']
['Team_Attributes']


In [4]:
# selecting first five rows from player table
player = pd.read_sql_query(
    '''
        SELECT * FROM Player
        LIMIT 5
    ''', conn, index_col='id'
)
player

,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
id,,,,,,
1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [5]:
# selecting first five rows from match table
match = pd.read_sql_query(
    '''
        SELECT * FROM Match
        LIMIT 5
    ''', conn, index_col='id'
)
match

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
id,,,,,,,,,,,,,,,,,,,,,
1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [6]:
# selecting first five rows from league table
league = pd.read_sql_query(
    '''
        SELECT * FROM League
        LIMIT 5
    ''', conn, index_col='id'
)
league

,country_id,name
id,,
1,1,Belgium Jupiler League
1729,1729,England Premier League
4769,4769,France Ligue 1
7809,7809,Germany 1. Bundesliga
10257,10257,Italy Serie A


In [7]:
# selecting first five rows from country table
country = pd.read_sql_query(
    '''
        SELECT * FROM Country
        LIMIT 5
    ''', conn, index_col='id'
)
country

,name
id,
1,Belgium
1729,England
4769,France
7809,Germany
10257,Italy


In [8]:
# selecting first five rows from team table
team = pd.read_sql_query(
    '''
        SELECT * FROM Team
        LIMIT 5
    ''', conn, index_col='id'
)
team

,team_api_id,team_fifa_api_id,team_long_name,team_short_name
id,,,,
1,9987,673,KRC Genk,GEN
2,9993,675,Beerschot AC,BAC
3,10000,15005,SV Zulte-Waregem,ZUL
4,9994,2007,Sporting Lokeren,LOK
5,9984,1750,KSV Cercle Brugge,CEB


In [9]:
# Number teams each country have
query = pd.read_sql_query(
    '''
        SELECT 
              c.name AS Country,
              COUNT(DISTINCT(team_long_name)) AS 'No. of Teams'
              FROM Match AS m
              LEFT JOIN Country AS c
              ON m.country_id = c.id
              LEFT JOIN Team AS t 
              ON m.home_team_api_id = t.team_api_id
              GROUP BY Country
    ''', conn
)
query

,Country,No. of Teams
0,Belgium,24
1,England,34
2,France,35
3,Germany,30
4,Italy,32
5,Netherlands,25
6,Poland,22
7,Portugal,29
8,Scotland,17
9,Spain,33


In [10]:
# total goals scored by home team grouped by country and Season
query1 = pd.read_sql_query(
    '''
        SELECT c.name AS Country,
               m.season AS Season,
               SUM(m.home_team_goal) AS 'Home Goal',
               SUM(m.away_team_goal) AS 'Away Goal'
        FROM Match as m 
        LEFT JOIN country AS c
        ON m.country_id = c.id
        GROUP BY Country, Season    
        ORDER BY Country     
    ''', conn
)
query1

,Country,Season,Home Goal,Away Goal
0,Belgium,2008/2009,499,356
1,Belgium,2009/2010,308,257
2,Belgium,2010/2011,382,253
3,Belgium,2011/2012,421,270
4,Belgium,2012/2013,375,328
...,...,...,...,...
83,Switzerland,2011/2012,235,190
84,Switzerland,2012/2013,272,190
85,Switzerland,2013/2014,290,230
86,Switzerland,2014/2015,289,228


In [11]:
# goals scored by each team grouped by season
query2 = pd.read_sql_query(
    '''
        SELECT t.team_long_name AS Team, 
               m.season as Season,
               SUM(m.home_team_goal) AS 'Home Goal',
               SUM(m.away_team_goal) AS 'Away Goal'
        FROM Match AS m
        LEFT JOIN Team AS t
        ON m.home_team_api_id = t.team_api_id 
        GROUP BY Team, Season
        ORDER BY Team
    ''', conn
)
query2

,Team,Season,Home Goal,Away Goal
0,1. FC Kaiserslautern,2010/2011,25,19
1,1. FC Kaiserslautern,2011/2012,12,28
2,1. FC Köln,2008/2009,14,25
3,1. FC Köln,2009/2010,18,29
4,1. FC Köln,2010/2011,30,21
...,...,...,...,...
1473,Śląsk Wrocław,2011/2012,26,14
1474,Śląsk Wrocław,2012/2013,23,15
1475,Śląsk Wrocław,2013/2014,21,16
1476,Śląsk Wrocław,2014/2015,21,9


In [12]:
query3 = pd.read_sql_query(
    '''
        SELECT c.name AS Country,
               t.team_long_name AS Team,
               m.season AS Season,
               SUM(m.home_team_goal) AS 'Home Goal',
               SUM(m.away_team_goal) AS 'Away Goal'
        FROM Match as m
        LEFT JOIN Country AS c
        ON m.country_id = c.id
        LEFT JOIN Team AS t
        ON m.home_team_api_id = t.team_api_id
        GROUP BY Country, Team, Season
        ORDER BY Country
    ''', conn
)
query3

,Country,Team,Season,Home Goal,Away Goal
0,Belgium,Beerschot AC,2008/2009,32,19
1,Belgium,Beerschot AC,2009/2010,20,22
2,Belgium,Beerschot AC,2010/2011,14,10
3,Belgium,Beerschot AC,2011/2012,26,13
4,Belgium,Beerschot AC,2012/2013,20,34
...,...,...,...,...,...
1473,Switzerland,Neuchâtel Xamax,2009/2010,35,27
1474,Switzerland,Neuchâtel Xamax,2010/2011,23,35
1475,Switzerland,Neuchâtel Xamax,2011/2012,12,10
1476,Switzerland,Servette FC,2011/2012,23,27


In [13]:
# count of matches won, lost and tie
query4 = pd.read_sql_query(
    '''
    WITH sub_q AS (
        SELECT     
                   c.name AS Country,
                   t.team_long_name AS Team,
                   COUNT(CASE WHEN m.home_team_goal > away_team_goal THEN 'Win' END) AS Won,
                   COUNT(CASE WHEN m.home_team_goal < away_team_goal THEN 'Lost' END) AS Lost,
                   COUNT(CASE WHEN m.home_team_goal = away_team_goal THEN 'Tie' END) AS Tie
         FROM Match AS m
         LEFT JOIN Country AS c
         ON m.country_id = c.id
         LEFT JOIN Team as t
         ON m.home_team_api_id = t.team_api_id
         GROUP BY Country, Team
         ORDER BY Country
         )
         SELECT ROW_NUMBER() OVER(ORDER BY Won DESC) AS 'Row Number',
                Country, 
                Team, 
                Won, 
                Lost,
                Tie
        FROM sub_q
    ''', conn, index_col='Row Number'
    )
query4

,Country,Team,Won,Lost,Tie
Row Number,,,,,
1,Spain,FC Barcelona,131,9,12
2,Spain,Real Madrid CF,129,12,11
3,Scotland,Celtic,120,9,23
4,England,Manchester United,116,20,16
5,England,Manchester City,113,21,18
...,...,...,...,...,...
292,Germany,DSC Arminia Bielefeld,2,7,8
293,Germany,SV Darmstadt 98,2,9,6
294,Scotland,Dunfermline Athletic,1,11,7


In [14]:
# total home goals scored grouped by country and team
query5 = pd.read_sql_query(
    '''
        SELECT name AS Name,
            team_long_name AS Team,
            --STRFTIME('%Y', date) AS Year, 
            SUM(home_team_goal) AS Goal
        FROM Match AS m
        LEFT JOIN Country as c
        ON m.country_id = c.id
        LEFT JOIN Team AS t
        ON m.home_team_api_id = t.team_api_id  
        GROUP BY Name, Team
        ORDER BY Goal DESC
        
    ''', conn
)
query5

,Name,Team,Goal
0,Spain,Real Madrid CF,505
1,Spain,FC Barcelona,495
2,Scotland,Celtic,389
3,Germany,FC Bayern Munich,382
4,Netherlands,PSV,370
...,...,...,...
291,France,AC Arles-Avignon,14
292,Portugal,Trofense,14
293,Portugal,Feirense,13
294,Spain,Córdoba CF,12


In [15]:
# total goals scored grouped by country and year
query6 = pd.read_sql_query(
    '''
        WITH year_wise AS (
            SELECT country_id, STRFTIME('%Y', date) AS Year, home_team_goal AS Goal
            FROM Match
        )
        SELECT name AS Name, Year , SUM(Goal) AS Goals
        FROM year_wise as y
        LEFT JOIN Country as c
        ON y.country_id = c.id
        GROUP BY Name, Year
        
    ''', conn
)
query6

,Name,Year,Goals
0,Belgium,2008,262
1,Belgium,2009,452
2,Belgium,2010,371
3,Belgium,2011,362
4,Belgium,2012,454
...,...,...,...
94,Switzerland,2012,222
95,Switzerland,2013,286
96,Switzerland,2014,289
97,Switzerland,2015,313


In [16]:
# extracting Year, Month, Week and day from date
query7 = pd.read_sql_query(
    '''
        SELECT date AS Date,
               STRFTIME('%Y', date) AS Year,
               STRFTIME('%m', date) AS Month,
               STRFTIME('%w', date) AS Week,
               STRFTIME('%d', date) AS Day
        FROM Match
    ''', conn
)
query7

,Date,Year,Month,Week,Day
0,2008-08-17 00:00:00,2008,08,0,17
1,2008-08-16 00:00:00,2008,08,6,16
2,2008-08-16 00:00:00,2008,08,6,16
3,2008-08-17 00:00:00,2008,08,0,17
4,2008-08-16 00:00:00,2008,08,6,16
...,...,...,...,...,...
25974,2015-09-22 00:00:00,2015,09,2,22
25975,2015-09-23 00:00:00,2015,09,3,23
25976,2015-09-23 00:00:00,2015,09,3,23
25977,2015-09-22 00:00:00,2015,09,2,22


In [17]:
# count of matches won, lost and tie
query8 = pd.read_sql_query(
    '''
    WITH sub_q AS (
        SELECT     
                   c.name AS Country,
                   season AS Season,
                   t.team_long_name AS Team,
                   --m.home_team_goal AS home_goal,
                   --m.away_team_goal AS away_goal,
                   COUNT(CASE WHEN m.home_team_goal > away_team_goal THEN 'Win' END) AS Won,
                   COUNT(CASE WHEN m.home_team_goal < away_team_goal THEN 'Lost' END) AS Lost,
                   COUNT(CASE WHEN m.home_team_goal = away_team_goal THEN 'Tie' END) AS Tie
         FROM Match AS m
         LEFT JOIN Country AS c
         ON m.country_id = c.id
         LEFT JOIN Team as t
         ON m.home_team_api_id = t.team_api_id
         GROUP BY Country, Season
         ORDER BY Country
         )
         SELECT ROW_NUMBER() OVER(ORDER BY Won DESC) AS 'Row Number',
                Country, 
                Season,
                Won, 
                Lost,
                Tie
        FROM sub_q
    ''', conn, index_col='Row Number'
    )
query8

,Country,Season,Won,Lost,Tie
Row Number,,,,,
1,Spain,2010/2011,197,104,79
2,Spain,2009/2010,194,91,95
3,England,2009/2010,193,91,96
4,Italy,2008/2009,192,93,95
5,Spain,2012/2013,189,107,84
...,...,...,...,...,...
84,Switzerland,2015/2016,80,53,47
85,Switzerland,2014/2015,76,56,48
86,Switzerland,2010/2011,72,57,51


In [18]:
# partitioning by Season and Country
query9 = pd.read_sql_query(
    '''
    WITH sub_q AS (
        SELECT 
              season AS Season,
              name AS Country,
              team_long_name AS Team,
              SUM(home_team_goal) AS Goal
        FROM Match AS m
        LEFT JOIN Country as c
        ON m.country_id = c.id
        LEFT JOIN Team AS t
        ON m.home_team_api_id = t.team_api_id  
        GROUP BY Season, Name, Team
        ORDER BY Goal DESC
        )
        SELECT 
              ROW_NUMBER() OVER(PARTITION BY Season, Country) AS 'Row No.',
              Season,
              Country,
              Team, 
              Goal
        FROM sub_q
        
    ''', conn, index_col='Row No.'
)
query9

,Season,Country,Team,Goal
Row No.,,,,
1,2008/2009,Belgium,RSC Anderlecht,48
2,2008/2009,Belgium,Standard de Liège,42
3,2008/2009,Belgium,Club Brugge KV,37
4,2008/2009,Belgium,Beerschot AC,32
5,2008/2009,Belgium,KAA Gent,31
...,...,...,...,...
6,2015/2016,Switzerland,FC Zürich,29
7,2015/2016,Switzerland,FC Thun,26
8,2015/2016,Switzerland,FC St. Gallen,25


In [19]:
# concatenating team name with country name
query10 = pd.read_sql_query(
    '''
    WITH sub_q AS (
        SELECT  
              team_long_name || ' (' || name || ')' AS Team,
              SUM(home_team_goal) AS Goal
        FROM Match AS m
        LEFT JOIN Country as c
        ON m.country_id = c.id
        LEFT JOIN Team AS t
        ON m.home_team_api_id = t.team_api_id  
        GROUP BY Season, Name, Team
        ORDER BY Goal DESC
        )
        SELECT 
              Team, 
              Goal
        FROM sub_q
        
    ''', conn
)
query10


,Team,Goal
0,FC Barcelona (Spain),73
1,Real Madrid CF (Spain),70
2,Real Madrid CF (Spain),70
3,Chelsea (England),68
4,Real Madrid CF (Spain),67
...,...,...
1473,Lechia Gdańsk (Poland),9
1474,KAA Gent (Belgium),6
1475,Lierse SK (Belgium),5
1476,Waasland-Beveren (Belgium),4


In [20]:
import plotly.express as px
import matplotlib.pyplot as plt 

#plt.rcParams['font.size'] = (10, 6)

df = query4[query4['Won'] > 0]

fig = px.treemap(
    df,
    path = ['Country', 'Team'],
    values='Won',
    color='Won',
    color_continuous_scale='RdBu',
    hover_data=['Won', 'Lost', 'Tie'],
    template='plotly_dark',
    title='Total Matches Won'
)
fig.show()

In [21]:
fig = px.bar(
    query8,
    x='Country', y='Won',
    title='Total Matches Won Country wise',
    color_discrete_map={
        '2009/2010': 'rgb(0, 0, 128)',
        '2010/2011': 'rgb(235, 207, 52)'},
        color='Season'
        #template='plotly_dark'
        )

fig.update_layout({
    'showlegend': True,
    'legend': {
        'title': 'All Seasons',
        'x': 1.01, 'y': 0.99,
        'bgcolor': 'rgb(246, 228, 129)',
         'font': {'color': 'black'}
    }
})
fig.update_xaxes(title_text='Countries')
fig.update_yaxes(title_text='Matches Won')
